# Monte Carlo π con Ray vs sequenziale

In [ ]:

import ray, random, time
ray.shutdown(); ray.init()

@ray.remote
def monte_carlo_simulation(num_samples:int):
    inside = 0
    rnd = random.Random()  # thread-local
    for _ in range(num_samples):
        x = rnd.uniform(-1,1); y = rnd.uniform(-1,1)
        if x*x + y*y <= 1: inside += 1
    return inside

def estimate_pi_ray(total_samples:int=2_000_000, num_workers:int=4):
    per = total_samples // num_workers
    futures = [monte_carlo_simulation.remote(per) for _ in range(num_workers)]
    inside = sum(ray.get(futures))
    return 4 * inside / total_samples

def estimate_pi_seq(total_samples:int=2_000_000):
    inside = 0
    rnd = random.Random()
    for _ in range(total_samples):
        x = rnd.uniform(-1,1); y = rnd.uniform(-1,1)
        if x*x + y*y <= 1: inside += 1
    return 4 * inside / total_samples

total = 2_000_000  # aumenta per test più pesanti
nw = max(1, int(ray.available_resources().get("CPU", 1)))
t0 = time.time(); pi_r = estimate_pi_ray(total, min(nw, 8)); t1 = time.time()
t2 = time.time(); pi_s = estimate_pi_seq(total); t3 = time.time()
print(f"Ray: {pi_r:.6f} in {t1-t0:.2f}s | Seq: {pi_s:.6f} in {t3-t2:.2f}s")
ray.shutdown()
